
# <div style="text-align: center; background-color: red; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">📊EDA |Earthquakes  Data | Visualization </div>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# <div style="text-align: center; background-color: red; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">1. Import Necessary Libraries</div>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import missingno as mno
import plotly.offline as pyo 
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
color_pal = sns.color_palette()
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')

import nltk
import re

import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

#Model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
df=pd.read_csv("/kaggle/input/the-ultimate-earthquake-dataset-from-1990-2023/Eartquakes-1990-2023.csv")



<a id="1"></a>
# <div style="text-align: center; background-color: red; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">2. Exploratory Data Analysis 📊</div>

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

In [ ]:
unique_values = df.nunique()
typpe_values = df.dtypes

summary_df = pd.DataFrame({'Unique Values': unique_values, 'Data Type': typpe_values})
print(summary_df)

In [ ]:
df.shape


# <div style="text-align: center; background-color: red; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">3. Null values</div>

In [ ]:
df.isna().sum()


# <div style="text-align: center; background-color: red; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">4. Duplicate rows</div>

In [ ]:
# Finding duplicate rows
duplicate_rows = df[df.duplicated(keep='first')]

# Number of duplicate rows
num_duplicates = duplicate_rows.shape[0]

# Displaying the duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
duplicate_rows

In [ ]:
# Drop duplicates rows
df.drop_duplicates(keep='first', inplace=True)

In [ ]:
state_counts = df['state'].value_counts()
print(state_counts.head(20))

In [ ]:
# Convert the 'date' column to datetime data type
df['date'] = pd.to_datetime(df['date'])
# Convert the 'time' column to time data type
df['time'] = pd.to_datetime(df['time']).dt.time

In [ ]:
# Filter earthquakes with magnitudo greater than 7
filtered_magnitudo = df[df['magnitudo'] > 7]

In [ ]:
# Filter earthquakes in California 
filtered_California = df[df['state'] == 'California']

In [ ]:
# Filter earthquakes in Alaska 
filtered_Alaska= df[df['state'] == 'Alaska']

In [ ]:
# Filter earthquakes in Nevada
filtered_Nevada = df[df['state'] == 'Nevada']


In [ ]:
# Filter earthquakes in Hawaii 
filtered_Washington = df[df['state'] == 'Washington']

In [ ]:
df.head()


# <div style="text-align: center; background-color: red; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">Time Series</div>

In [ ]:
def transform_date(df):
    df = df.copy()
    df["hour"] = df.date.dt.hour
    df["day_of_month"] = df.date.dt.day
    df["weekday"] = df.date.dt.day_name()
    df["month"] = df.date.dt.month
    df["month_name"] = df.date.dt.month_name()
    df["year"] =df.date.dt.year
    return df
df = transform_date(df)


# <div style="text-align: center; background-color: red; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">5. Data visualisation</div>

In [ ]:
# Import the ProfileReport class from the ydata_profiling library
from ydata_profiling import ProfileReport

# Create a comprehensive profile report for the DataFrame 'df'
# This report will contain various statistics, insights, and visualizations about the data
profile = ProfileReport(df)

In [ ]:
profile                     # or this one
profile.to_notebook_iframe() # use this line to show the output

In [ ]:
import pandas as pd
import plotly.express as px

# Assuming you have a DataFrame named 'df'

# Columns for which you want to create plots
columns_to_plot = ['state', 'place','data_type','month_name','status','year']

for column in columns_to_plot:
    top10_values = df[column].value_counts()[:10]
    
    # Create a bar plot using Plotly Express
    fig = px.bar(
        y=top10_values.values,
        x=top10_values.index,
        color=top10_values.index,
        color_discrete_sequence=px.colors.sequential.PuBuGn,
        text=top10_values.values,
        title=f'Top 10 {column}',
        template='plotly_white'
    )
    fig.update_layout(
        title_text=f'{column} Distribution',
        height=700,
        xaxis_title=column,
        yaxis_title="Count",
        font=dict(size=17, family="Franklin Gothic")
    )
    
    # Display the plot
    fig.show()

In [ ]:
# Select the top 10 magnitudes in the DataFrame
top10_magnitudes = df['magnitudo'].sort_values(ascending=False).head(10)

# Create a bar plot using Plotly Express
# y-axis: Magnitude values, x-axis: Index of the top 10 magnitudes,
# color-coded by the index, and using a color palette
fig = px.bar(
    y=top10_magnitudes.values,  # y-values: top 10 magnitudes
    x=top10_magnitudes.index,   # x-values: indices of the top 10 magnitudes
    color=top10_magnitudes.index,  # Color the bars based on the indices
    #color_discrete_sequence=px.colors.sequential.PuBuGn,  # Set color palette
    text=top10_magnitudes.values,  # Display the magnitude values on top of the bars
    title='Top 10 Magnitudes',  # Set the title of the plot
    template='plotly_white'  # Use a dark template for the plot
)

# Update the layout of the plot
fig.update_layout(
    title_text='Magnitude Distribution',  # Set the title of the plot
    height=700,  # Set the height of the plot
    xaxis_title="Index",  # Label for the x-axis
    yaxis_title="Magnitude",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:

# Columns and y-labels for which you want to create plots
columns_to_plot = [
    ('date', 'magnitudo', 'magnitudo by date'),
    ('time', 'magnitudo', 'magnitudo by time'),
    ('state', 'magnitudo', 'magnitudo by state')
]

for column, y_label, title in columns_to_plot:
    data = filtered_magnitudo.groupby(column)[y_label].sum().reset_index()
    
    # Create a line plot using Plotly Express
    fig = px.line(
        data,
        x=column,
        y=y_label,
        title=title,
        height=800,
        color_discrete_sequence=px.colors.qualitative.Pastel1  # Change the color palette
    )
    fig.update_layout(
        xaxis_title=column,
        yaxis_title=y_label,
        font=dict(size=17, family="Franklin Gothic")
    )
    
    # Display the plot
    fig.show()


In [ ]:

# Columns and y-labels for which you want to create plots
columns_to_plot = [
    ('date', 'magnitudo', 'magnitudo by date'),
    ('time', 'magnitudo', 'magnitudo by time')
]

for column, y_label, title in columns_to_plot:
    data = filtered_California.groupby(column)[y_label].sum().reset_index()
    
    # Create a line plot using Plotly Express
    fig = px.line(
        data,
        x=column,
        y=y_label,
        title=title,
        height=800,
        color_discrete_sequence=px.colors.qualitative.Pastel1  # Change the color palette

    )
    fig.update_layout(
        xaxis_title=column,
        yaxis_title=y_label,
        font=dict(size=17, family="Franklin Gothic")
    )
    
    # Display the plot
    fig.show()


In [ ]:

# Columns and y-labels for which you want to create plots
columns_to_plot = [
    ('date', 'magnitudo', 'magnitudo by date'),
    ('time', 'magnitudo', 'magnitudo by time')
]

for column, y_label, title in columns_to_plot:
    data = filtered_Nevada.groupby(column)[y_label].sum().reset_index()
    
    # Create a line plot using Plotly Express
    fig = px.line(
        data,
        x=column,
        y=y_label,
        title=title,
        height=800
    )
    fig.update_layout(
        xaxis_title=column,
        yaxis_title=y_label,
        font=dict(size=17, family="Franklin Gothic")
    )
    
    # Display the plot
    fig.show()


In [ ]:
import plotly.express as px

def plot_average_magnitude(data, group_col, x_label, title):
    magnitude_by_group = data.groupby(group_col)['magnitudo'].mean().reset_index()

    fig = px.line(magnitude_by_group, x=group_col, y='magnitudo',
                  title=title,
                  color_discrete_sequence=px.colors.qualitative.Pastel1,  # Change the color palette
                  labels={group_col: x_label, 'magnitudo': 'Average Magnitude'},
                  template='plotly_white')

    fig.show()

# Assuming df is your DataFrame containing earthquake data
# For year
plot_average_magnitude(df, 'year', 'Year', 'Average Magnitude of Earthquakes by Year')

# For month
plot_average_magnitude(df, 'month', 'Month', 'Average Magnitude of Earthquakes by Month')

# For month_name
plot_average_magnitude(df, 'month_name', 'Month Name', 'Average Magnitude of Earthquakes by Month')

# For day_of_month
plot_average_magnitude(df, 'day_of_month', 'Day of Month', 'Average Magnitude of Earthquakes by Day')


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo

# Assuming you have a DataFrame named 'df'

# Columns and y-labels for which you want to create plots
columns_to_plot = [
    ('place', 'Count   place by magnitudo', 'magnitudo'),
    ('state', 'Count  state by magnitudo', 'magnitudo')
]

# Initialize Plotly for notebook integration
pyo.init_notebook_mode(connected=True)

for column, title, y_label in columns_to_plot:
    # Calculate the average for the current column and sort
    average_data = df.groupby(column)[y_label].mean().reset_index()
    average_data = average_data.sort_values(by=y_label, ascending=False)
    
    # Select the top 10 items
    top_10_data = average_data.head(10)
    
    # Create a new figure using Plotly
    fig = go.Figure()
    
    # Add a bar trace to the figure
    fig.add_trace(go.Bar(
        x=top_10_data[column],
        y=top_10_data[y_label],
        marker_color='red'  # Set the color of the bars
    ))
    
    # Update the layout and appearance of the plot
    fig.update_layout(
        title=title,
        xaxis_title=column,
        yaxis_title=y_label,
        template='plotly_white',
        font=dict(color='black'),
        height=800
    )
    
    # Display the plot
    pyo.iplot(fig)

In [ ]:
df.head()

In [ ]:
# Columns and titles for which you want to create sunburst charts
columns_to_plot = [
    ('status', 'Distribution of status'),
    ('month_name', 'Distribution of month_name'),
    ('data_type','Distribution of data_type')
]

for column, title in columns_to_plot:
    # Create a sunburst chart using Plotly Express
    fig = px.sunburst(df, path=[column], color_discrete_sequence=px.colors.qualitative.Set2)
    fig.update_layout(title_text=title, height=500)
    
    # Display the plot
    fig.show()

In [ ]:
def plot_strongest(df):
    gb_top = df.groupby(["state"])\
               .agg({"significance":"max", "magnitudo":"max"})\
               .reset_index()

    b = df[["year", "state","significance","longitude", "latitude","magnitudo"]]
    merge = pd.merge(gb_top, b, how = "left", on =["state","significance", "magnitudo"]).sort_values(by="year")


    fig = px.scatter_geo(merge, 
                         lat="latitude", lon="longitude", 
                         size="significance",
                         animation_frame="year",
                         projection="natural earth",
                         title = "Strongest eartquake by year" ,
                         template='plotly_white'  # Use a dark template for the plot
)
    fig.show()
    
plot_strongest(df)


# <div style="text-align: center; background-color: red; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">6. Categorical</div>

In [ ]:
# Select columns with object (categorical) data types
num_cols = df.select_dtypes(include='object').columns.tolist()

# Initialize the LabelEncoder
le = LabelEncoder()

# Apply Label Encoding to the selected numerical columns
for x in num_cols:  
    df[x] = le.fit_transform(df[x])

# Now, your categorical columns (excluding column 0) have been converted to numerical values
df.head()


# <div style="text-align: center; background-color: red; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">7.HeatMap & Most Positively & Most Negatively</div>

In [ ]:
Corr_Matrix = df.corr()

# Set up the figure and plot the heatmap
plt.figure(figsize=(25,25))
sns.heatmap(Corr_Matrix, annot=True, cmap='BuPu', center=0)
plt.show()

# Most Positively

In [ ]:
print('Top 5 Most Positively Correlated to the To magnitudo')
Corr_Matrix['magnitudo'].sort_values(ascending=False).head(5)

# Most Negatively

In [ ]:
print('Top 5 Most Positively Correlated to magnitudo ')
Corr_Matrix['magnitudo'].sort_values(ascending=True).head(5)